In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import numpy as np
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import yfinance as yf
from datetime import datetime
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV


**Comparision between the models**

import statements, loading the Featuremart and stock file and making sure data has opening and closing market values

At the end of this, data_frame is a clean, merged dataset that includes:

Feature-engineered indicators from your CSV.

Historical stock price data (Open, Close, etc.).

All aligned on date index.

In [ ]:
# preparing data
# Set display options for better visualization
pd.set_option('display.max_columns', None)
np.set_printoptions(precision=4)

# Define date range
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 12, 31)

# Load and prepare features
feature = pd.read_csv('GS_FeatureMart_New.csv')
feature['Date'] = pd.to_datetime(feature['Date'])
feature = feature.set_index('Date')

# Fetch target stock data
stock_symbol = 'GS'
stock = yf.download(stock_symbol, start_date, end_date)

if isinstance(stock.columns, pd.MultiIndex):
    # Create a new DataFrame with flattened column names
    flattened_cols = [f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col for col in stock.columns]
    stock_flat = stock.copy()
    stock_flat.columns = flattened_cols
else:
    stock_flat = stock.copy()

# Now merge the DataFrames properly
data_frame = pd.merge(feature, stock_flat, left_index=True, right_index=True, how='inner')
if 'Close' not in data_frame.columns and 'Adj Close' in data_frame.columns:
    data_frame['Close'] = data_frame['Adj Close']
elif 'Close' not in data_frame.columns:
    # Look for column that might contain 'Close'
    close_cols = [col for col in data_frame.columns if 'Close' in col]
    if close_cols:
        data_frame['Close'] = data_frame[close_cols[0]]
    else:
        raise ValueError("No Close price column found in data")

if 'Open' not in data_frame.columns:
    open_cols = [col for col in data_frame.columns if 'Open' in col]
    if open_cols:
        data_frame['Open'] = data_frame[open_cols[0]]
    else:
        raise ValueError("No Open price column found in data")



<ipython-input-7-f3f46dcf65f1>:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  feature['Date'] = pd.to_datetime(feature['Date'])
[*********************100%***********************]  1 of 1 completed


**Rolling Window Backtesting Logic**

Trading Signal:
If the model predicts the next day’s price will rise → Buy at Open, Sell at Close.
If the model predicts a drop → Short at Open, Cover at Close.

selected_features: Feature Set for Predictive Modeling

This list includes key technical indicators, macroeconomic rates, and volatility measures that are used as input features to the machine learning models.

In the model training and backtesting process[,](https://) this list is critical for:

Standardizing Inputs: Ensures all features are on the same scale before model training.

Model Inputs: Defines exactly what the model "sees" to learn and make predictions.

Backtest Consistency: Guarantees consistent features are used across rolling windows.

Feature Importance Analysis: Only these variables are evaluated later for importance ranking.


In [ ]:
# Fill missing values and reset index
data_frame = data_frame.ffill()
data_frame.reset_index(inplace=True)

selected_features = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'SP500', 'T10YIE',
       'DEXUSUK', 'DEXUSEU', 'T10Y3M', 'T5YIE', 'DBAA', 'DAAA',
       'BAMLH0A0HYM2EY', 'BAMLH0A0HYM2', 'AMERIBOR', 'T10Y2Y', 'DGS30', 'EFFR',
       'SMA_10', 'RSI_15', 'BBL_10_2.0', 'BBP_10_2.0', 'OBV']

# Running backtest with different models to compare it with those
def run_backtest(model_name, model, data_frame, window_size=252, position_size=184, initial_balance=50000):
    # Create a copy of the data frame
    data = data_frame.copy()

    # Standardize features
    scaler = StandardScaler()
    data[selected_features] = scaler.fit_transform(data[selected_features])

    # Set parameters
    horizon = 1
    start = pd.Timestamp('2021-02-01')

    # Find the index of the start date or closest date after it
    valid_dates = data['Date'] >= start
    if valid_dates.any():
        train_start = valid_dates.idxmax()
    else:
        print(f"No dates found after {start}. Using the last {window_size} days instead.")
        train_start = max(0, len(data) - window_size - 30)  # Leave 30 days for testing

    print(f"Training starts at index {train_start}, date: {data.iloc[train_start]['Date']}")

    y_pred, y_test = {}, {}
    position = position_size
    signal, gain_loss, balance = {}, {}, {}
    balance[train_start] = initial_balance

    # Iterate through trading days
    for today in range(train_start, len(data) - 1):
        yesterday = today - 1
        tomorrow = today + 1

        if yesterday < 0 or tomorrow >= len(data):
            continue  # Skip if indices are out of bounds

        # Retrieve training data
        X_train = data.iloc[max(0, today - window_size):today][selected_features]
        y_train = data['Close'].iloc[max(0, today - window_size):today]

        # Test data
        X_test = data.iloc[today:today+1][selected_features]

        # Fit model and make prediction
        if len(X_train) >= 30:
            model.fit(X_train, y_train)
            y_pred[tomorrow] = model.predict(X_test)[0]
            y_test[tomorrow] = data['Open'].iloc[tomorrow]

            # Generate trading signal
            if y_pred[tomorrow] > data['Close'].iloc[today]:
                signal[tomorrow] = 1  # Buy (predict price will rise)
                # Buy at open, sell at close
                gain_loss[tomorrow] = position * (data['Close'].iloc[tomorrow] - data['Open'].iloc[tomorrow])
            else:
                signal[tomorrow] = -1  # Sell/Short (predict price will fall)
                # Short at open, cover at close
                gain_loss[tomorrow] = position * (data['Open'].iloc[tomorrow] - data['Close'].iloc[tomorrow])


            balance[tomorrow] = balance[today] + gain_loss[tomorrow]

    # Remove first entry which is just the initial balance
    if train_start in balance:
        del balance[train_start]

    # Create results dataframe
    signal_df = pd.DataFrame.from_dict(signal, orient='index', columns=['signal'])
    gain_loss_df = pd.DataFrame.from_dict(gain_loss, orient='index', columns=['gain_loss'])
    balance_df = pd.DataFrame.from_dict(balance, orient='index', columns=['balance'])

    results = pd.concat([signal_df, gain_loss_df, balance_df], axis=1)

    if len(results) == 0:
        print(f"No results for {model_name}. Check date range and data availability.")
        return None, None

    # Add date information
    results_with_date = results.copy()
    results_with_date['Date'] = pd.to_datetime(data['Date'].iloc[results.index].values)
    # Tracks what would happen if you simply held the stock instead of trading.


    # Calculate buy and hold performance
    initial_price = data['Close'].iloc[train_start]
    results_with_date['buy_hold'] = initial_balance - position * initial_price + position * data['Close'].iloc[results.index].values

    # Calculate performance metrics
    strategy_return = (results_with_date['balance'].iloc[-1] - initial_balance) / initial_balance
    buyhold_return = (results_with_date['buy_hold'].iloc[-1] - initial_balance) / initial_balance

    # Calculate daily returns for Sharpe ratio
    daily_returns = results_with_date['balance'].pct_change().dropna()
    sharpe_ratio = (daily_returns.mean() / daily_returns.std()) * np.sqrt(252) if daily_returns.std() != 0 else 0

    # Calculate max drawdown
    running_max = results_with_date['balance'].cummax()
    drawdown = (results_with_date['balance'] / running_max - 1)
    max_drawdown = drawdown.min()
    # Measures worst % drop from peak equity – important for risk management.


    # Calculate win rate
    win_days = (results_with_date['gain_loss'] > 0).sum()
    total_days = len(results_with_date)
    win_rate = win_days / total_days if total_days > 0 else 0

    metrics = {
        'Strategy Return': strategy_return,
        'Buy & Hold Return': buyhold_return,
        'Outperformance': strategy_return - buyhold_return,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown,
        'Win Rate': win_rate,
        'Total Trades': total_days
    }
# win_rate = (# of positive days) / (total trade days)

    return results_with_date, metrics

# Define models to test
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    # Reduce XGBoost complexity for faster training and inherent handling of outliers
    "XGBoost": xgb.XGBRegressor(n_estimators=50, max_depth=3, random_state=42),
    "SVR": SVR(C=1.0, epsilon=0.2)
}

# Run backtests for each model
results_dict = {}
metrics_dict = {}

for model_name, model in models.items():
    print(f"Running backtest for {model_name}...")
    results, metrics = run_backtest(model_name, model, data_frame.copy())
    if results is not None and metrics is not None:
        results_dict[model_name] = results
        metrics_dict[model_name] = metrics


Running backtest for LinearRegression...
Training starts at index 272, date: 2021-02-01 00:00:00
Running backtest for Ridge...
Training starts at index 272, date: 2021-02-01 00:00:00
Running backtest for Lasso...
Training starts at index 272, date: 2021-02-01 00:00:00
Running backtest for RandomForest...
Training starts at index 272, date: 2021-02-01 00:00:00
Running backtest for XGBoost...
Training starts at index 272, date: 2021-02-01 00:00:00
Running backtest for SVR...
Training starts at index 272, date: 2021-02-01 00:00:00


In [ ]:

# Convert metrics to DataFrame for easy comparison
if metrics_dict:
    metrics_df = pd.DataFrame.from_dict(metrics_dict, orient='index')
    print("\nModel Performance Comparison:")
    print(metrics_df)

    # Sort models outperformance
    if 'Outperformance' in metrics_df.columns:
        metrics_df_sorted = metrics_df.sort_values('Outperformance', ascending=False)
        print("\nModels sorted by Outperformance:")
        print(metrics_df_sorted)

        if results_dict:
            # Plot 1: All models vs buy-and-hold
            first_model = list(results_dict.keys())[0]
            buy_hold_line = results_dict[first_model]['buy_hold']

            fig = go.Figure()

            for model_name, results in results_dict.items():
                fig.add_trace(go.Scatter(
                    x=results['Date'],
                    y=results['balance'],
                    mode='lines',
                    name=f'{model_name} Strategy',
                    hovertemplate='<b>Date</b>: %{x}<br><b>Balance</b>: $%{y:.2f}<extra></extra>'
                ))

            # Add buy and hold line
            fig.add_trace(go.Scatter(
                x=results_dict[first_model]['Date'],
                y=buy_hold_line,
                mode='lines',
                line=dict(dash='dash', color='black'),
                name='Buy and Hold',
                hovertemplate='<b>Date</b>: %{x}<br><b>Value</b>: $%{y:.2f}<extra></extra>'
            ))

            # Add range selector buttons and slider
            fig.update_layout(
                title='GS Day Trading Strategies vs Buy-and-Hold',
                xaxis=dict(
                    rangeselector=dict(
                        buttons=list([
                            dict(count=1, label="1m", step="month", stepmode="backward"),
                            dict(count=3, label="3m", step="month", stepmode="backward"),
                            dict(count=6, label="6m", step="month", stepmode="backward"),
                            dict(count=1, label="YTD", step="year", stepmode="todate"),
                            dict(count=1, label="1y", step="year", stepmode="backward"),
                            dict(step="all")
                        ])
                    ),
                    rangeslider=dict(visible=True),
                    type="date"
                ),
                yaxis=dict(title='Portfolio Value ($)'),
                hovermode='closest',
                legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
            )

            fig.show()

            if len(metrics_df_sorted) > 0:
                best_model = metrics_df_sorted.index[0]

                # Calculate cumulative returns for detailed analysis
                for model_name, results in results_dict.items():
                    # Add daily returns and cumulative returns
                    results['daily_returns'] = results['balance'].pct_change()
                    results['cumulative_returns'] = (1 + results['daily_returns'].fillna(0)).cumprod() - 1

                    # Same for buy and hold
                    results['bh_daily_returns'] = results['buy_hold'].pct_change()
                    results['bh_cumulative_returns'] = (1 + results['bh_daily_returns'].fillna(0)).cumprod() - 1

                # Plot 3: Cumulative return comparison
                fig3 = go.Figure()

                for model_name, results in results_dict.items():
                    fig3.add_trace(go.Scatter(
                        x=results['Date'],
                        y=results['cumulative_returns'],
                        mode='lines',
                        name=f'{model_name}',
                        hovertemplate='<b>Date</b>: %{x}<br><b>Return</b>: %{y:.2%}<extra></extra>'
                    ))

                # Add buy and hold line
                fig3.add_trace(go.Scatter(
                    x=results_dict[first_model]['Date'],
                    y=results_dict[first_model]['bh_cumulative_returns'],
                    mode='lines',
                    line=dict(dash='dash', color='black'),
                    name='Buy and Hold',
                    hovertemplate='<b>Date</b>: %{x}<br><b>Return</b>: %{y:.2%}<extra></extra>'
                ))

                # Add range selector buttons and slider
                fig3.update_layout(
                    title='GS Cumulative Returns Comparison',
                    xaxis=dict(
                        rangeselector=dict(
                            buttons=list([
                                dict(count=1, label="1m", step="month", stepmode="backward"),
                                dict(count=3, label="3m", step="month", stepmode="backward"),
                                dict(count=6, label="6m", step="month", stepmode="backward"),
                                dict(count=1, label="YTD", step="year", stepmode="todate"),
                                dict(count=1, label="1y", step="year", stepmode="backward"),
                                dict(step="all")
                            ])
                        ),
                        rangeslider=dict(visible=True),
                        type="date"
                    ),
                    yaxis=dict(title='Cumulative Return (%)', tickformat='.0%'),
                    hovermode='closest',
                    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                )

                fig3.show()

                # Print summary statistics for the best model
                print(f"\nPerformance Summary for Best Model ({best_model}) on Goldman Sachs (GS):")
                print(f"Total Return: {metrics_dict[best_model]['Strategy Return']:.2%}")
                print(f"Buy & Hold Return: {metrics_dict[best_model]['Buy & Hold Return']:.2%}")
                print(f"Outperformance: {metrics_dict[best_model]['Outperformance']:.2%}")
                print(f"Sharpe Ratio: {metrics_dict[best_model]['Sharpe Ratio']:.2f}")
                print(f"Max Drawdown: {metrics_dict[best_model]['Max Drawdown']:.2%}")
                print(f"Win Rate: {metrics_dict[best_model]['Win Rate']:.2%}")
                print(f"Total Trades: {metrics_dict[best_model]['Total Trades']}")

                # Analyze profitable and unprofitable days
                best_results = results_dict[best_model]
                profitable_days = (best_results['gain_loss'] > 0).sum()
                unprofitable_days = (best_results['gain_loss'] <= 0).sum()
                avg_profit_days = best_results.loc[best_results['gain_loss'] > 0, 'gain_loss'].mean() if profitable_days > 0 else 0
                avg_loss_days = best_results.loc[best_results['gain_loss'] <= 0, 'gain_loss'].mean() if unprofitable_days > 0 else 0

                print("\nDaily Statistics:")
                print(f"Profitable Days: {profitable_days} ({profitable_days/(profitable_days+unprofitable_days):.2%})")
                print(f"Unprofitable Days: {unprofitable_days} ({unprofitable_days/(profitable_days+unprofitable_days):.2%})")
                print(f"Average Profit on Winning Days: ${avg_profit_days:.2f}")
                print(f"Average Loss on Losing Days: ${avg_loss_days:.2f}")

                if unprofitable_days > 0 and profitable_days > 0:
                    profit_factor = abs(avg_profit_days * profitable_days / (avg_loss_days * unprofitable_days))
                    print(f"Profit Factor: {profit_factor:.2f}")


else:
    print("No valid results were generated for any models. Check your data and parameters.")


Model Performance Comparison:
                  Strategy Return  Buy & Hold Return  Outperformance  \
LinearRegression         0.795158           1.188674       -0.393516   
Ridge                   -0.062942           1.188674       -1.251616   
Lasso                    0.525924           1.188674       -0.662750   
RandomForest            -0.708121           1.188674       -1.896794   
XGBoost                  0.492331           1.188674       -0.696343   
SVR                     -0.188679           1.188674       -1.377353   

                  Sharpe Ratio  Max Drawdown  Win Rate  Total Trades  
LinearRegression      0.804832     -0.310422  0.523374           984  
Ridge                 0.202788     -0.677473  0.491870           984  
Lasso                 0.586676     -0.324511  0.510163           984  
RandomForest          0.131633     -0.922046  0.482724           984  
XGBoost               0.558292     -0.461041  0.509146           984  
SVR                   0.042887     -0.


Performance Summary for Best Model (LinearRegression) on Goldman Sachs (GS):
Total Return: 79.52%
Buy & Hold Return: 118.87%
Outperformance: -39.35%
Sharpe Ratio: 0.80
Max Drawdown: -31.04%
Win Rate: 52.34%
Total Trades: 984

Daily Statistics:
Profitable Days: 515 (52.34%)
Unprofitable Days: 469 (47.66%)
Average Profit on Winning Days: $677.64
Average Loss on Losing Days: $-659.33
Profit Factor: 1.13


| Component        | Type                     | Formula / Logic                                              | Purpose                                                        |
|------------------|--------------------------|---------------------------------------------------------------|----------------------------------------------------------------|
| **Rolling Window** | Algorithm                | Train on past `N` days, predict next day, roll forward        | Simulates real-world trading (walk-forward validation)         |
| **Signal Generation** | Decision Logic         | Trading Signal: If the model predicts the next day’s price will rise → Buy at Open, Sell at Close. If the model predicts a drop → Short at Open, Cover at Close.           | Converts model output into actionable trading signals          |
| **Sharpe Ratio** | Financial Metric          |Sharpe Ratio = (Mean Daily Return − Mean Risk-Free Rate) / Standard Deviation of Daily Returns × √252 | Measures return per unit of volatility (risk-adjusted return) |
| **Max Drawdown** | Time Series Metric        |MaxDrawdown=Minimum of (Current Value ÷ Previous Peak−1)| Measures worst portfolio loss from a peak (risk exposure)     |
| **Buy & Hold**   | Baseline Strategy         | \(\{Return} = \{Final Price} - \{Initial Price}\) | Benchmark for evaluating active strategies                     |
| **Gain / Loss**  | Arithmetic                | \({PnL} = ({Close} - {Open}) \times {Position Size}\) | Computes profit/loss per trade                                |
